In [75]:
import pandas as pd 
from pathlib import Path

N5_BASE = "outputs/0_final_results/MATH-full_dt.math/chatgpt1106/model_selection_prompts/n5_baseline.jsonl"
N5_RIMS_PATHS = list(Path("outputs/0_final_results").glob("MATH-full_dt.math/chatgpt1106/*/n5_rims_*.jsonl"))

df_base = pd.read_json(N5_BASE, lines=True)
rims_df_d = {"/".join(path.parts[-2:]): pd.read_json(path, lines=True) for path in N5_RIMS_PATHS}

df_base.shape, {k: v.shape for k, v in rims_df_d.items()}   

((4996, 19),
 {'rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.5.jsonl': (4991, 21),
  'rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.3.jsonl': (4995, 21),
  'rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.1.jsonl': (4995, 21),
  'rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.6.jsonl': (4996, 21),
  'rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.2.jsonl': (4988, 21),
  'rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.5.jsonl': (4996, 21),
  'rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.3.jsonl': (4996, 21),
  'rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.1.jsonl': (4995, 21),
  'rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.6.jsonl': (4996, 21),
  'rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.2.jsonl': (4989, 21)})

In [76]:
def check(majority_ans, majvote_ans):
    # did majority_ans came from majvote_ans?
    majvote_submitted = False
    if isinstance(majvote_ans, list):
        if isinstance(majority_ans, list):
            for a in majority_ans:
                if a in majvote_ans:
                    majvote_submitted = True
                    break
        elif majority_ans is not None:
            majvote_submitted = (majority_ans in majvote_ans)
        else:
            raise DONOTREACHHERE
    else:
        majvote_submitted = False
    return majvote_submitted

In [77]:
def analyze_selection_effect(df, return_flag=False):
    selection_runs = df.majvote_ans.apply(lambda lst: lst.count(None)>0 if isinstance(lst, list) else False)
    majvote_ans_are_submitted = df.apply(lambda row: check(row.majority_ans, row.majvote_ans), axis=1)  & selection_runs
    selection_ans_changed_answer = df.apply(lambda row: check(row.majority_ans, row.candid_answers), axis=1) & (~majvote_ans_are_submitted) & selection_runs
    na = df.majority_ans.isna() | df.majvote_ans.isna() | df.candid_answers.isna() | df.majority_ans.apply(lambda x: x==[])
    res = {
            "total_rows": len(df),
            "selection_occurred": selection_runs, #.sum(),
            "majvote_ans_are_submitted": majvote_ans_are_submitted, # .sum(),
            "selection_ans_changed_final_submission": selection_ans_changed_answer, #.sum(),
            "na_or_majority_ans_emptylist": na.sum(),
            # "diff": (majvote_ans_are_submitted | selection_ans_changed_answer) & (~selection_runs), # majvote_ans_are_submitted.sum() + selection_ans_changed_answer.sum() - selection_runs.sum(),
            # "edge_cases": (selection_runs &  (~majvote_ans_are_submitted) & (~selection_ans_changed_answer)),
        }
    if not return_flag:
        for key in ["selection_occurred", "majvote_ans_are_submitted", "selection_ans_changed_final_submission", ]: # diff, edge_cases
            res[key] = res[key].sum()

    return res

In [78]:
from pprint import pprint

res_base = analyze_selection_effect(df_base)
res_rims_d = {k: analyze_selection_effect(v) for k, v in rims_df_d.items()}
print("base")
pprint(res_base)
sort_rims = sorted(list(res_rims_d.keys()), key=lambda txt: txt.split("/")[-1])

for k in sort_rims:
    pprint(k)
    pprint(res_rims_d[k])

res_base_flags = analyze_selection_effect(df_base, return_flag=True)
res_rims_d_flags = {k: analyze_selection_effect(v, return_flag=True) for k, v in rims_df_d.items()}

df_base[df_base.majority_ans.apply(lambda a: a==[])].loc[:, ["candid_answers", "answer", "majority_ans"]]


base
{'majvote_ans_are_submitted': 2459,
 'na_or_majority_ans_emptylist': 336,
 'selection_ans_changed_final_submission': 980,
 'selection_occurred': 3775,
 'total_rows': 4996}
'rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.1.jsonl'
{'majvote_ans_are_submitted': 2187,
 'na_or_majority_ans_emptylist': 26,
 'selection_ans_changed_final_submission': 1561,
 'selection_occurred': 3773,
 'total_rows': 4995}
'rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.1.jsonl'
{'majvote_ans_are_submitted': 2155,
 'na_or_majority_ans_emptylist': 23,
 'selection_ans_changed_final_submission': 1596,
 'selection_occurred': 3772,
 'total_rows': 4995}
'rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.2.jsonl'
{'majvote_ans_are_submitted': 2501,
 'na_or_majority_ans_emptylist': 308,
 'selection_ans_changed_final_submission': 958,
 'selection_occurred': 3767,
 'total_rows': 4988}
'rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.2.jsonl'
{'majvote_ans_are_submitted': 2481,
 'na_or_majority_ans_empt

,candid_answers,answer,majority_ans
44,"[$\boxed{\frac{1}{12}}$, $\boxed{\frac{6}{7}}$...",\frac{1}{12},[]
108,"[$(1,6)$, $(6,6)$, None, $\boxed{(9, 5)}$, $\n...","(2,12)",[]
133,"[18.0, $5+3\sqrt{5}$, $\n\nFinal Answer: $, $\...",18,[]
160,"[\( \frac{243}{8} \), \(\frac{729}{64}\), $\fr...",\frac{243}{8},[]
200,"[9, $\boxed{0}$, $\boxed{5}$, 100.0, $\boxed{1...",29,[]
...,...,...,...
4983,"[$\boxed{\left(\frac{1}{\sqrt{2}}, -\frac{1}{2...","\left( \frac{1}{\sqrt{2}}, \frac{1}{\sqrt{2}} ...",[]
4985,[\[\mathbf{b} = \begin{pmatrix} 4 \\ 7 \end{pm...,\begin{pmatrix} 2/5 \\ -1/5 \end{pmatrix},[]
4986,"[$ and $, $\n\nSince the angle between vectors...",55^\circ,[]
4990,"[$\boxed{\text{All real numbers}}$, \(x\), $\b...","7, -2, \text{ and } -5",[]


In [80]:
keys = sort_rims

corrects = [
2549,
2495,
2486,
2530,
2516,
2502,
2526,
2538,
2548,
2516,
2533,
]

lucks = [ 
    res_rims_d[k]["na_or_majority_ans_emptylist"] for k in sort_rims
]


diff_from_bases = [corrects[0]-i for i in corrects[1:]]
luck_surpluses = [l - lucks[0] for l in lucks[1:]]

for d, l, name in zip(diff_from_bases, luck_surpluses, sort_rims):
    print(d, l, name)

54 -3 rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.1.jsonl
63 282 rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.1.jsonl
19 262 rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.2.jsonl
33 -14 rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.2.jsonl
47 -15 rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.3.jsonl
23 302 rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.3.jsonl
11 308 rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.5.jsonl
1 -4 rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.5.jsonl
33 -17 rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.6.jsonl


In [ ]:
# txt1 prompt 에 좀더 희망이 있어보임
# txt prompt는 greedy에서 어떤 결과를 보였었나?

# chatgpt 1106, gpt4turbo 모두 txt1 prompt결과가 더 일관성 있고 좋았다.
# 그러므로 txt1 prompt에 대해서 `get_concordant_answer_n(candid_answers) == []` 인 경우, candid_answers 에서 하나 고르도록 하지 말고, selection answers 중에서 하나 고르도록 고쳐보자. (이 편이 차이를 부각하기 더 좋다. candid에서 샘플링할 경우 majvote에 의해서 결과가 희석될 것.) 


# diff to catchup / lucks surplus / rims prompts+temp
# 19 262 rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.2.jsonl
# 33 -14 rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.2.jsonl


# 11 308 rims_math_p2c-cot.pal-p2c.pal-cot__.txt1/n5_rims_T0.5.jsonl
# 1 -4 rims_math_p2c-cot.pal-p2c.pal-cot__.txt/n5_rims_T0.5.jsonl

In [ ]:
### lucky punch failed
MATH_RIMS1 (*.txt1 prompt) slightly worse than baseline again (2 problems), others are worse than that. scoring improvement improved baseline too, and the trend did not change

next: prompt tweak--> test on T0 n1